<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Pretrained Segmentation Model
</p>


In [ ]:
from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit.segment import segment_vessels
from fundus_vessels_toolkit.utils.data_io import load_image

vscode_theme()

In [2]:
# Path to the raw fundus image
RAW_PATH = "PATH/TO/FUNDUS/IMAGE"

# Path to a vessel ground truth (for comparison purposes)
VESSELS_PATH = "PATH/TO/VESSEL/GROUND/TRUTH"

### Load Image


In [3]:
raw = load_image(RAW_PATH)

vessels_true = load_image(VESSELS_PATH, binarize=True)

### Segment vessels


When called, `fundus_vessels_toolkit.segment.segment_vessels()` will download the weights of a pretrained model, instanciate the model, convert the provided image to the appropriate tensor format, pass it to the model and return the prediction converted back to numpy format.


In [4]:
vessels = segment_vessels(raw)

For a finner control, the torch pretrained model may also be retreived with `fundus_vessels_toolkit.models.segmentation_model( model_name )`.


### Precompute fundus mask

`segment_vessels()` will compute the mask of the fundus image to clean any prediction from the image background. You can precompute or fine-tune the mask using `fundus_vessels_toolkit.segment.fundus_ROI`.

This is especially useful when you want to speed up the vessels segmentation of a series of images with the same fundus mask.

In [5]:
from fundus_vessels_toolkit.segment import fundus_ROI

mask = fundus_ROI(raw)
vessels = segment_vessels(raw, roi_mask=mask)

### Display Segmentation Result


Displays the predicted segmentation and the ground truth side by side.

_(The prediction is on the left, the ground truth on the right, the middle show the false positive in green and the false negative in orange.)_


In [ ]:
m = Mosaic(3, background=raw, cell_height=800, cols_titles=["Prediction", "Pred vs GT", "GT"])
m[0].add_label(vessels, "vessels", colormap="white", opacity=0.3)
m[0].add_label(1 - mask, "fundus_ROI", colormap="magenta", opacity=0.3)
m[1].add_label(vessels + 2 * vessels_true, "vessels", colormap={3: "white", 1: "green", 2: "orange"})
m[2].add_label(vessels_true, "vessels", colormap="white", opacity=0.3)

m.show()